## Basics & Linear Regression

### What is Pytorch Tensor?

In [43]:
import torch
import numpy as np

In [44]:
t1 = torch.tensor(2.,requires_grad=True)
t2 = torch.tensor([1, 2, 3.],requires_grad=True)
t3 = torch.tensor([[1,2.,3],[4,5,6]],requires_grad=True)
t4 = torch.tensor([[[1,2.,3],[1,2,3]],
                    [[1,2,3],[1,2,3]],
                    [[1,2,3],[1,2,3]]],requires_grad=True)

print(f"Shapes of tensors are {t1.shape}, {t2.shape}, {t3.shape}, {t4.shape}")

Shapes of tensors are torch.Size([]), torch.Size([3]), torch.Size([2, 3]), torch.Size([3, 2, 3])


In [45]:
w , b = 0.8 , 0.1
z = w * t1 + b
z.backward()
print(f"The gradient w.r.t. t1 is given by - {t1.grad}")
print(t2.grad)

The gradient w.r.t. t1 is given by - 0.800000011920929
None


### Converting back & forth to Numpy

In [46]:
"""
Import numpy array as tensor, helpful as this uses the same memory location of the np array
"""
np_arr = np.array([[1,2,3],[4,5,6]])
y = torch.from_numpy(np_arr)
print(y,y.dtype)

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32) torch.int32


## Linear Regression

Let us explore the Pytorch functionality with an artificial example. Suppose, we have a dataset where we are given the produce of Apples and Oranges for different regions with data on rainfall, temperature and humidity. 

In [47]:
# Import and visualize data
import pandas as pd
data = pd.read_excel("data.xlsx")
data.head()

,Region,Temp(F),Rainfall(mm),Humidity(%),Apples(ton),Oranges (ton)
0,Kanto,73,67,43,56,70
1,Johto,91,88,64,81,101
2,Hoenn,87,134,58,119,133
3,Sinnoh,102,43,37,22,37
4,Unova,69,96,70,103,119


In [48]:
# Define the input and target tensors. Target tensors are correct values which will help to train our model
input_tensor = torch.tensor(data.iloc[:,1:4].values,dtype=float)
targets_tensor = torch.tensor(data.iloc[:,4:6].values,dtype=float)
print(f"Input tensor is {input_tensor}")
print(f"Output tensor is: {targets_tensor}")

Input tensor is tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]], dtype=torch.float64)
Output tensor is: tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]], dtype=torch.float64)


In [49]:
# Initialize random weights and biases when starting to train

w = torch.randn(targets_tensor.shape[1],input_tensor.shape[1],requires_grad=True)
b = torch.randn(targets_tensor.shape[1],requires_grad=True)
print(w,b)

tensor([[-0.8927, -1.7226, -1.4668],
        [ 0.4105, -1.0740,  0.6396]], requires_grad=True) tensor([-0.1545, -1.1236], requires_grad=True)


In [50]:
# Define a function which returns a linear model for regression

def model(x):
    return x @ w.t().double() + b.double()

preds = model(input_tensor.double())
print(preds)

tensor([[-243.8158,  -15.6106],
        [-326.8643,  -17.3437],
        [-393.7335,  -72.2265],
        [-219.5613,   18.2328],
        [-329.8060,  -31.1296]], dtype=torch.float64, grad_fn=<AddBackward0>)


In [51]:
# Define mean-squared-error as the loss function
def mse(pred,targets):
    diff = pred-targets
    return torch.sum(torch.mul(diff,diff))

loss_mse = mse(preds,targets_tensor)
print(loss_mse)

tensor(851154.7560, dtype=torch.float64, grad_fn=<SumBackward0>)


In [52]:
# Backpropate the loss/error and calculate gradients w.r.t. each element of the weight matrix
loss_mse.backward()
print(w.grad)

tensor([[-316225.7812, -353245.0312, -215936.2500],
        [ -94293.5078, -117739.8750,  -68723.6953]])


In [53]:
# Update the weights as per a learning rate (alpha)
alpha = 1e-3

with torch.no_grad():
    w -= w.grad * alpha 
    b -= b * alpha
    w.grad.zero_()
    b.grad.zero_()

print(w,b)

tensor([[315.3330, 351.5224, 214.4694],
        [ 94.7040, 116.6659,  69.3633]], requires_grad=True) tensor([-0.1543, -1.1225], requires_grad=True)


In [54]:
pred = model(input_tensor.double())
loss = mse(pred,targets_tensor)
print(loss)

tensor(2.6446e+10, dtype=torch.float64, grad_fn=<SumBackward0>)
